#  Que peuvent faire les *transformers* ?

Installez la bibliothèque 🤗 *Transformers* pour exécuter ce *notebook*.

L'outil le plus basique de la bibliothèque 🤗 *Transformers* est la fonction `pipeline()`. Elle relie un modèle avec ses étapes de pré-traitement et de post-traitement, permettant d'entrer n'importe quel texte et d'obtenir une réponse compréhensible :

In [9]:
%pip install "transformers[torch]"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
You should consider upgrading via the '/Users/ashnaahmad/Documents/GitHub/course/.env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
from transformers import pipeline

In [11]:
classifier = pipeline("sentiment-analysis")
classifier(
    "I've been waiting for a HuggingFace course my whole life."
)  # J'ai attendu un cours d'HuggingFace toute ma vie.

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598049521446228}]

classifier = pipeline("application")
classifier("sentence", "sentence") to run multiple sentences through the application

Par défaut, ce pipeline sélectionne un modèle pré-entraîné qui a été spécifiquement entraîné pour l'analyse de sentiment en anglais. Le modèle est téléchargé et mis en cache lorsque vous créez l'objet `classifier`. Si vous réexécutez la commande, c'est le modèle mis en cache qui sera utilisé et il n'y a pas besoin de télécharger le modèle à nouveau.

Il y a trois étapes principales lorsque vous passez du texte à un pipeline :

1. le texte est prétraité pour qu'il ait un format compréhensible par le modèle,
2. les données prétraitées sont passées au modèle,
3. les prédictions du modèle sont post-traitées de sorte que vous puissiez les comprendre.


Voici une liste non-exhaustive des [pipelines disponibles](https://huggingface.co/transformers/main_classes/pipelines.html) :

- `feature-extraction` (pour obtenir la représentation vectorielle d'un texte)
- `fill-mask`
- `ner` (*named entity recognition* ou reconnaissance d'entités nommées en français)
- `question-answering`
- `sentiment-analysis`
- `summarization`
- `text-generation`
- `translation`
- `zero-shot-classification`

Regardons de plus près certains d'entre eux !

### Analyse de sentiments

i guess a particular model as well as a particular application is also a parameter of pipeline (see below). also when using a specific model, make sure it uses the right framework (pt/tf)

In [12]:
classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9598049521446228}]

In [14]:
classifier = pipeline("sentiment-analysis", model="philschmid/pt-tblard-tf-allocine")
classifier("J'ai attendu un cours d'HuggingFace toute ma vie.") 

[{'label': 'NEGATIVE', 'score': 0.8035174608230591}]

Intéressant ! On observe que le résultat est négatif là où pour la version en anglais le résultat est positif.

In [ ]:
classifier(
    ["J'ai attendu un cours d'HuggingFace toute ma vie.", 
     "Je déteste tellement ça !"]
) # pour classifier plusieurs phrases

La phrase "J'ai attendu un cours d'HuggingFace toute ma vie." qui était précedemment négative devient à présent positive.

### Zéro shot classification

Nous allons commencer par nous attaquer à une tâche plus difficile où nous devons classer des textes qui n'ont pas été annotés. C'est un scénario très répandu dans les projets réels car l'annotation de textes est généralement longue et nécessite parfois une expertise dans un domaine. Pour ce cas d'usage, le pipeline `zero-shot-classification` est très puissant : il vous permet de **spécifier les labels** à utiliser pour la classification, de sorte que **vous n'ayez pas à vous soucier des labels du modèle pré-entraîné**. Nous avons déjà vu comment le modèle peut classer un texte comme positif ou négatif en utilisant ces deux labels mais **il peut également classer le texte en utilisant n'importe quel autre ensemble de labels que vous souhaitez.**

In [15]:
classifier = pipeline("zero-shot-classification", model="BaptisteDoyen/camembert-base-xnli") # set model
classifier(
    "C'est un cours sur la bibliothèque Transformers",
    candidate_labels=["éducation", "politique", "affaires"], # set custom models for classification, which will be assigned based on training (learned association of words with labels)
)

{'sequence': "C'est un cours sur la bibliothèque Transformers",
 'labels': ['éducation', 'affaires', 'politique'],
 'scores': [0.8710299134254456, 0.09802455455064774, 0.030945533886551857]}

### Génération de texte

Maintenant, nous allons voir comment utiliser un pipeline pour générer du texte. L'idée principale ici est que vous fournissez seulement un extrait de texte qui **va être complété par du texte généré automatiquement** par le modèle. Cette fonction est similaire à la fonction de texte prédictif que l'on trouve sur de nombreux téléphones portables. La génération de texte implique de l'aléatoire, donc il est normal que vous n'obteniez pas les mêmes résultats que ceux présentés ci-dessous.

Vous pouvez améliorer votre recherche de modèle (model=) en cliquant sur les *filtres* de langue et choisir un modèle qui génère du texte dans une autre langue. Le *Hub* contient également des *checkpoints* pour des modèles multilingues qui supportent plusieurs langues.

Une fois que vous avez choisi un modèle, vous verrez que vous pouvez tester son fonctionnement en ligne directement. Cela vous permet de tester rapidement les capacités du modèle avant de le télécharger.

Tous les modèles peuvent être testé directement depuis votre navigateur en utilisant l'API d'inférence qui est disponible sur le site [Hugging Face](https://huggingface.co/). Vous pouvez jouer avec le modèle directement sur sa page en entrant du texte personnalisé et en regardant le modèle traiter les données d'entrée.

In [ ]:
generator = pipeline("text-generation", model="asi/gpt-fr-cased-small")
generator("# Dans ce cours, nous vous enseignerons comment")

In [ ]:
generator = pipeline("text-generation", model="asi/gpt-fr-cased-small")
generator(
    "# Dans ce cours, nous vous enseignerons comment",
    max_length=30,  #parameters
    num_return_sequences=1,
)

### Remplacement des mots masqués

L'argument `top_k` permet de contrôler le nombre de possibilités que vous souhaitez afficher. Notez que dans ce cas, le modèle **remplace le mot spécial `<mask>`**, qui est souvent appelé un **mot masqué**. D'autres modèles permettant de remplacer les mots manquants peuvent avoir des mots masqués différents, donc il est toujours bon de vérifier le mot masqué approprié lorsque vous comparez d'autres modèles. Une façon de le vérifier est de regarder le mot masqué utilisé dans l'outil de test de la page du modèle.

In [16]:
unmasker = pipeline("fill-mask", model="camembert-base")
unmasker(" Ce cours vous apprendra tout sur les modèles <mask>.", top_k=2)

[{'score': 0.107318215072155,
  'token': 15328,
  'token_str': 'automobiles',
  'sequence': 'Ce cours vous apprendra tout sur les modèles automobiles.'},
 {'score': 0.07520466297864914,
  'token': 4007,
  'token_str': 'électriques',
  'sequence': 'Ce cours vous apprendra tout sur les modèles électriques.'}]

### Reconnaissance d'entités nommées

In [ ]:
ner = pipeline("ner", model="Jean-Baptiste/camembert-ner", grouped_entities=True)
ner("Je m'appelle Sylvain et je travaille à Hugging Face à Brooklyn.")

Nous pouvons voir que le modèle a correctement identifié Sylvain comme une personne (PER), Hugging Face comme une organisation (ORG) et Brooklyn comme un lieu (LOC).

Il est possible d'utiliser l'option `grouped_entities=True` lors de la création du pipeline pour **regrouper les parties du texte qui correspondent à la même entité : ici le modèle à correctement regroupé `Hugging` et `Face` comme une seule organisation**, même si le nom comporte plusieurs mots. En effet, comme nous allons voir dans le prochain chapitre, la prétraitement du texte sépare parfois certains mots en plus petites parties. Par exemple, `Sylvain` est séparé en quatre morceaux : `S`, `##yl`, `##va`, et `##in`. Dans l'étape de post-traitement, le pipeline a réussi à regrouper ces morceaux.

### Réponse à des questions

In [ ]:
question_answerer = pipeline("question-answering", model="etalab-ia/camembert-base-squadFR-fquad-piaf")
question_answerer(
    question="Où est-ce que je travaille ?",
    context="Je m'appelle Sylvain et je travaille à Hugging Face à Brooklyn.",
)

###  Résumé

In [ ]:
summarizer = pipeline("summarization", model="moussaKam/barthez-orangesum-abstract")
summarizer(
    """
    L'Amérique a changé de façon spectaculaire au cours des dernières années. Non seulement le nombre de 
    diplômés dans les disciplines traditionnelles de l'ingénierie telles que le génie mécanique, civil, 
    l'électricité, la chimie et l'aéronautique a diminué, mais dans la plupart 
    des grandes universités américaines, les programmes d'études d'ingénierie se concentrent désormais sur 
    et encouragent largement l'étude des sciences de l'ingénieur. Par conséquent, il y a 
    de moins en moins d'offres dans les sujets d'ingénierie traitant de l'infrastructure, 
    l'environnement et les questions connexes, et une plus grande concentration sur les sujets de haute 
    technologie, qui soutiennent en grande partie des développements scientifiques de plus en plus 
    complexes. Si cette dernière est importante, elle ne doit pas se faire au détriment
    de l'ingénierie plus traditionnelle.

    Les économies en développement rapide telles que la Chine et l'Inde, ainsi que d'autres 
    pays industrialisés d'Europe et d'Asie, continuent d'encourager et de promouvoir
    l'enseignement de l'ingénierie. La Chine et l'Inde, respectivement, diplôment 
    six et huit fois plus d'ingénieurs traditionnels que les États-Unis. 
    Les autres pays industriels maintiennent au minimum leur production, tandis que l'Amérique 
    souffre d'une baisse de plus en plus importante du nombre de diplômés en ingénierie
    et un manque d'ingénieurs bien formés.
"""
)

###  Traduction

In [ ]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-fr")
translator("This course is produced by Hugging Face.")